In [11]:
#All parameters to modify (in operation order):
#--------------------Function Parameters---------------------------------#
#Selection inputs
layer = "line"                                            # The Line layer whose features will be selected
rasterInput = "FL_GulfCoast_T_ProjectRaster"              # Map Elevation Data layer
lineInput = "line_Select"

#Selection output
outFeature = r"E:\Projects\ViewshedAnalysisOUR\viewshed.gdb\line_Select2"


#Point Generation input
pointSpacing = 75                                         # Percent distance for points

#Point generation output
pointOutput = r"E:\Projects\ViewshedAnalysisOUR\viewshed.gdb\GeneratePoints"

#Viewshed generation input
pointInput = "GeneratePoints"                            #*This should match pointOutput*
observerElevation = "6 Meters"                           # Height of viewshed observer point
viewDistance = "1 Miles"                                 # Maximum view distance of viewshed

#Viewshed final output!
outRaster = r"E:\Projects\ViewshedAnalysisOUR\viewshed.gdb\Viewshed_FL_G3"




#--------------------Feature Selection Group-------------------------#

def selected_features(layer):
    # Check if there is a selection
    if arcpy.Describe(layer).FIDSet:
        # Count the selected features
        result = arcpy.GetCount_management(layer)   
        count = int(result.getOutput(0))
        fids = arcpy.Describe(layer).FIDSet.split('; ')
    else:
        # No selection, return 0
        count = 0
    return fids

print(selected_features(layer))
row_list = selected_features(layer)

#generate a query from the selected features
queryString = ''
x = 0
for i in row_list:
    queryString += "OBJECTID = " + i
    x = x + 1
    try:
        if (row_list[x]):
            queryString += " Or "
    except IndexError:
        print("")

print(queryString)

#Create new layer from selection
arcpy.analysis.Select(
    in_features= layer,
    out_feature_class= outFeature,
    where_clause= queryString
)

['1']
﻿
OBJECTID = 1


<Result 'E:\\Projects\\ViewshedAnalysisOUR\\viewshed.gdb\\line_Select2'>

In [12]:
#------------------Point Generation-------------------#

# generate points along line
arcpy.management.GeneratePointsAlongLines(
    Input_Features= lineInput,
    Output_Feature_Class= pointOutput,
    Point_Placement="PERCENTAGE",
    Distance=None,
    Percentage=pointSpacing,
    Include_End_Points="NO_END_POINTS",
    Add_Chainage_Fields="NO_CHAINAGE"
)

<Result 'E:\\Projects\\ViewshedAnalysisOUR\\viewshed.gdb\\GeneratePoints'>

In [13]:
#---------------------Viewshed Generation-------------------#

#Special input; Viewshed environment variables. Coordinate system, projection, etc
#This might be easiest to set by just opening up the tool, entering the environment variables, and copying a new command to pull this line from 

with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_16N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAD_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', parallelProcessingFactor="100%", scratchWorkspace=r"E:\Projects\ViewshedAnalysisOUR\viewshed.gdb"):
    out_raster = arcpy.sa.Viewshed2(
        in_raster=rasterInput,
        in_observer_features=pointInput,
        out_agl_raster=None,
        analysis_type="OBSERVERS",
        vertical_error="0 Meters",
        out_observer_region_relationship_table=r"E:\Projects\ViewshedAnalysisOUR\viewshed.gdb\GeneratePoints_Viewshed2",
        refractivity_coefficient=0.13,
        surface_offset="0 Meters",
        observer_elevation=None,
        observer_offset=observerElevation,
        inner_radius=None,
        inner_radius_is_3d="GROUND",
        outer_radius=viewDistance,
        outer_radius_is_3d="GROUND",
        horizontal_start_angle=0,
        horizontal_end_angle=360,
        vertical_upper_angle=180,
        vertical_lower_angle=-180,
        analysis_method="ALL_SIGHTLINES",
        analysis_target_device="CPU_ONLY"
    )
    out_raster.save(outRaster)